In [1]:
import torch

def corr2d(x,k):
    '''计算二维互相关运算'''
    #获取卷积核的高（h）和宽（w）（行数与列数）
    h,w = k.shape
    #输出计算结果y的形状为[输入张量x的行数-h+1，输入张量x的列数-w+1]，同时初始化y的数值（全部为0）
    y = torch.zeros((x.shape[0] - h + 1, x.shape[1] - w + 1))
    #嵌套两层循环计算y对应位置的值并更新
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            #对应位置的值为x取i,j为起点的与卷积核同样形状的子张量，计算与卷积核的点积并累加
            y[i,j] = (x[i:i+h,j:j+w]*k).sum()

    return y

#实现多输入通道互相关运算
def corr2d_multi_in(X,K):
    return sum(corr2d(x,k) for x,k in zip(X,K))

#验证互相关运算的输出
X = torch.tensor([[[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]],
                 [[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]]])
K = torch.tensor([[[0.0,1.0],[2.0,3.0]],
                  [[1.0,2.0],[3.0,4.0]]])

print(corr2d_multi_in(X,K))

tensor([[ 56.,  72.],
        [104., 120.]])


In [2]:
#计算多个通道输出的互相关函数
def corr2d_multi_in_out(x,K):
    return torch.stack([corr2d_multi_in(x,k) for k in K],0)

K = torch.stack((K,K+1,K+2),0)
print(K.shape)
print(corr2d_multi_in_out(X,K))

torch.Size([3, 2, 2, 2])
tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])


In [3]:
#验证1*1卷积
def corr2d_multi_in_out_1x1(X,K):
    #获取x的形状
    c_i, h,w = X.shape
    #获取k的行数
    c_o = K.shape[0]
    X = X.reshape((c_i,h*w))
    K = K.reshape((c_o,c_i))
    Y = torch.matmul(K,X)
    return Y.reshape((c_o,h,w))

X = torch.normal(0,1,(3,3,3))
K = torch.normal(0,1,(2,3,1,1))

Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X,K)
assert float(torch.abs(Y1-Y2).sum()) < 1e-6

In [9]:
x=torch.normal(0,1,(3,4))
print(x)
print(x.shape[0])

tensor([[ 1.8136,  1.5060, -0.3767, -0.6224],
        [-0.0726,  0.6746, -0.7830, -0.0398],
        [-0.5180,  0.1446, -0.4937,  0.0917]])
3
